## <span style='color:#ff5f27'> 📝 Imports

In [2]:
import pandas as pd

from functions.functions import *

---

## <span style='color:#ff5f27'> 💽 Loading Historical Data</span>


#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [14]:
df_air_quality = pd.read_csv('data_poland/air_quality/air_quality_merged.csv')
df_air_quality.head()

date  pm10    o3  no2  so2  co           city  pm25
0  2023-1-2  23.0  18.0  NaN  3.0 NaN  bielsko biała   NaN
1  2023-1-3  25.0  27.0  7.0  2.0 NaN  bielsko biała   NaN
2  2023-1-4  19.0  24.0  4.0  1.0 NaN  bielsko biała   NaN
3  2023-1-5   9.0  20.0  3.0  1.0 NaN  bielsko biała   NaN
4  2023-1-6  13.0  17.0  NaN  2.0 NaN  bielsko biała   NaN

In [15]:
df_air_quality.city.unique()

array(['bielsko biała', 'cracow', 'poznań', 'gdańsk', 'wroclaw',
       'biała podlaska', 'katowice', 'warsaw'], dtype=object)

In [16]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time)
df_air_quality.sort_values(by = ['city','date'],inplace = True,ignore_index = True)

df_air_quality.head()

date  pm10    o3  no2  so2   co            city  pm25
0  1411077600000  45.0  34.0  3.0  4.0  4.0  biała podlaska   NaN
1  1411164000000   NaN  26.0  4.0  2.0  NaN  biała podlaska  99.0
2  1411250400000   NaN  32.0  NaN  1.0  NaN  biała podlaska   NaN
3  1411336800000   NaN  21.0  NaN  1.0  NaN  biała podlaska   NaN
4  1411423200000   NaN  23.0  NaN  1.0  NaN  biała podlaska   NaN

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [33]:
df_weather = pd.read_csv('data_poland/weather/weather_merged.csv')

df_weather.head(3)

city        date  tempmax  tempmin  temp  feelslikemax  \
0  bielsko biała  2022-01-01     10.7      8.3   9.1          10.7   
1  bielsko biała  2022-01-02     12.0      6.1   9.6          12.0   
2  bielsko biała  2022-01-03     10.5      7.1   8.6          10.5   

   feelslikemin  feelslike  dew  humidity  ...  snowdepth  windgust  \
0           5.4        6.6  8.3      95.1  ...       41.4      46.7   
1           4.8        9.2  5.2      75.1  ...       35.1      54.0   
2           3.0        4.9  3.2      69.3  ...       28.7      68.1   

   windspeed  winddir  cloudcover  visibility  solarradiation  solarenergy  \
0       32.2    245.6        99.0         7.4            11.6          1.0   
1       39.0    197.7        92.6        40.5            30.9          2.6   
2       35.8    230.0        83.1        46.2            25.5          2.2   

   uvindex              conditions  
0        1          Rain, Overcast  
1        2          Rain, Overcast  
2        2  Rain, Partially cloudy  

[3 rows x 24 columns]

In [34]:
df_weather.date = df_weather.date.apply(timestamp_2_time)
df_weather.sort_values(by=['city', 'date'],inplace=True, ignore_index=True)

df_weather = df_weather.dropna()

df_weather.head(3)

city           date  tempmax  tempmin  temp  feelslikemax  \
0  biała podlaska  1640991600000      9.4      2.7   6.6           6.8   
1  biała podlaska  1641078000000      8.0     -0.6   3.2           5.2   
2  biała podlaska  1641164400000      8.5      6.0   7.4           5.7   

   feelslikemin  feelslike  dew  humidity  ...  snowdepth  windgust  \
0          -0.5        3.6  4.6      87.9  ...        0.0      50.4   
1          -3.2        0.6  1.6      89.2  ...        0.0      39.6   
2           2.7        4.5  5.1      85.7  ...        0.0      57.6   

   windspeed  winddir  cloudcover  visibility  solarradiation  solarenergy  \
0       22.4    303.9        95.7        27.3            14.8          1.3   
1       18.7    200.3        90.5        21.6            15.4          1.2   
2       24.4    260.6        87.1        21.3            12.7          1.1   

   uvindex              conditions  
0        1          Rain, Overcast  
1        1          Rain, Overcast  
2        1  Rain, Partially cloudy  

[3 rows x 24 columns]

---

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [12]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) ID2223_Anton
	 (2) ID2223_Ernest

Enter project to access: 2

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5476
Connected. Call `.close()` to terminate connection gracefully.


---

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [24]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'poland_air_quality_fg',
        description = 'Air Quality characteristics of each day for Polish cities',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

air_quality_fg.insert(df_air_quality)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5476/fs/5383/fg/14949


Uploading Dataframe: 0.00% |          | Rows 0/22362 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5476/jobs/named/poland_air_quality_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f8aae3c7610>, None)

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [38]:
weather_fg = fs.get_or_create_feature_group(
        name = 'poland_weather_fg',
        description = 'Weather characteristics of each day for Polish cities',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

Uploading Dataframe: 0.00% |          | Rows 0/2922 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5476/jobs/named/poland_weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f8aae608ca0>, None)

Meta data

In [45]:
df_meta = pd.read_excel('data_poland/meta.xlsx')
df_meta['city'] = df_meta['city'].apply(lambda c: c.lower())
df_meta.drop(['city_id'], axis=1, inplace=True)
df_meta.head()

city        lon         lat    area  density  no_people
0  biała podlaska  52.033333   23.116667   49.40  1152.06      56942
1       białystok  53.135278   23.145556  102.13  2872.90     293413
2   bielsko-biała  49.822500   19.044444  124.51  1351.90     168319
3       bydgoszcz  53.125000   18.011111  175.98  1941.70     337666
4          gdańsk  54.347500   18.645278  263.44  1845.00     486022

In [ ]:
meta_fg = fs.get_or_create_feature_group(
        name = 'poland_meta_fg',
        description = 'characteristics of Polish cities',
        version = 1,
        primary_key = ['city'],
        online_enabled = True
    )    

meta_fg.insert(df_meta)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5476/fs/5383/fg/14952


Uploading Dataframe: 0.00% |          | Rows 0/21 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5476/jobs/named/poland_meta_fg_1_offline_fg_backfill/executions


---